In [1]:
import numpy as np
import time

In [12]:
class Table(object):
    def __init__(self, table=None, seed=None):
        # pre-defined soduku seed storehouse
        seeds_list = ['000300000400012083000004060002056007053700000000400006800000090015900000000008001',
                      '000720640076100003040805002090048000000000007000050080300002000000000005008006030',
                      '720000064000076000400000000005604100000000028100053006000500200340180000000000010',
                      '000000000590800700000021800037000000050790000000030180005002000810000040006080903',
                      '104000000000005000060080073000801900650000000000300008020030007000007130470000890',
                      '090300000000029001600000007070000010500000064000804009010200600780090000004050003',
                      '200080010040100300000090000000700840090021000006000000030000580100800007027053000',
                      '200600050080902000000040600000409006500000090090200140001000003040310000020000008',
                      '046903000003050060900002003005006000800000010010780200000000050081300007000800104',
                      '030010080000605000021903450087000520600020009014000760045701290000204000090030010',
                      '039000610700308004100090008010907040003020900040805020600040002300601009097000860',
                      '000020000007000900040983010002008600304000107001600500080732060009000800000060000',
                      '502090100000100080300006002040000700600000001005000090900700004060003000007020503',
                      '000000020030009006001047000000100470005000300027008000000530800800200060010000000',
                      '040006030700040001000800900001000008020030060300000100007004000100080007060300020',
                      '004000703800902000030000000089100000500000008000009260000000020000804005605000100',
                      '500000001900300046001400700004050000000060008000903060050010070000090050007008000',
                      '812753649903600000070090200050007000000045700000100030001000068008500010090000400'
                      ]

        # if new table is given, process it to a 9x9 soduku table
        if table is not None:

            # if table is of type str with 81 character
            if isinstance(table, str):
                sudoku = np.array(list(map(eval, ','.join(table).split(',')))).reshape(9,9)

            # if table is already a 9x9 list or numpy array
            if isinstance(table, (list, np.ndarray)):
                sudoku = np.array(table)

        # if no table is given, a random soduku table will be chosen from the seed storehouse
        else:
            length = len(seeds_list)
            if seed is None:
                seed = time.time()
            rng = round(seed)%length
            sudoku_seed = seeds_list[rng]
            sudoku = np.array(list(map(eval, ','.join(sudoku_seed).split(',')))).reshape(9,9)

        self.real = sudoku
        self.seed = seed
        self.draft = np.zeros((9,9,9))+1
        self.draft_units = [[self.draft[i*3:(i+1)*3, j*3:(j+1)*3]
                      for i in range(3)] for j in range(3)]
        self.real_units = [[self.real[i*3:(i+1)*3, j*3:(j+1)*3]
                      for i in range(3)] for j in range(3)]
        self.iter_cnt = 0
        self.guess_cnt = 0
        self.real_filter()

    def __str__(self):
        return str(self.real)+'\n'+'zero count: '+str(sum(sum(self.real==0)))

    def __eq__(self, other):
        return (self.real == other.real).all()

    def my_copy(self):
        new_table = Table(seed=self.seed)
        # print(id(new_table), id(self))
        # print(id(self.real), id(new_table.real))
        # print()
        new_table.real = self.real.copy()
        # print(id(self.real), id(new_table.real))
        # print()
        new_table.draft = self.draft.copy()
        new_table.iter_cnt = self.iter_cnt
        new_table.guess_cnt = self.guess_cnt
        return new_table
        
    def show_draft(self, num):
        print(self.draft[:,:,num-1])

    def real_filter(self):
        for i, row in enumerate(self.real):
            for j ,real in enumerate(row):
                if real > 0:
                    self.draft[i, j,:] = 0

    def simple_filter(self):
        for i, row in enumerate(self.real):
            for j ,real in enumerate(row):
                if real > 0:
                    self.draft[i, :, real-1] = 0  # row filter
                    self.draft[:, j, real-1] = 0  # col filter
        for m, unit_row in enumerate(self.real_units):  # unit filter
            for n, unit in enumerate(unit_row):
                for i, row in enumerate(unit):
                    for j, real in enumerate(row):
                        if real > 0:
                            self.draft_units[m][n][:,:,real-1] = 0

    def exclusive_fill(self):
        self.simple_filter()
        self.close_filter()
        self.straight_filter()
        for i, row in enumerate(self.real):
            for j, real in enumerate(row):
                if real == 0 and self.draft[i,j,:].sum() == 1:
                    layer_cnt = np.nonzero(self.draft[i,j,:].flatten())[0][0]
                    self.real[i, j] = layer_cnt+1
                    self.draft[i, j, :] = 0

    def unique_fill(self):
        self.real_filter()
        self.simple_filter()
        self.close_filter()
        self.straight_filter()
        # unit
        for m, unit_row in enumerate(self.draft_units):
            for n, unit in enumerate(unit_row):
                for layer_cnt in range(9):
                    if unit[:,:,layer_cnt].flatten().sum() == 1:
                        non_zero = np.nonzero(unit[:,:,layer_cnt])
                        self.real_units[m][n][non_zero[0][0], non_zero[1][0]] = layer_cnt + 1
                        test = self.inspect()
                        self.draft_units[m][n][non_zero[0][0], non_zero[1][0],:] = 0
                        self.draft_units[m][n][:,:,layer_cnt] = 0

        # row and col
        mapping = {0:np.array([0,0,0,0,0,0,0,0,0]),
                   1:np.array([1,0,0,0,0,0,0,0,0]),
                   2:np.array([0,1,0,0,0,0,0,0,0]),
                   3:np.array([0,0,1,0,0,0,0,0,0]),
                   4:np.array([0,0,0,1,0,0,0,0,0]),
                   5:np.array([0,0,0,0,1,0,0,0,0]),
                   6:np.array([0,0,0,0,0,1,0,0,0]),
                   7:np.array([0,0,0,0,0,0,1,0,0]),
                   8:np.array([0,0,0,0,0,0,0,1,0]),
                   9:np.array([0,0,0,0,0,0,0,0,1]),}

        # row
        copy_draft = self.draft.copy()
        add = np.array([[mapping[real] for real in row] for row in self.real])
        draft = copy_draft + add
        for m, row in enumerate(draft):
            for layer_cnt in range(9):
                if row[:, layer_cnt].sum() == 1:
                    position = np.nonzero(row[:, layer_cnt].flatten())[0][0]
                    if self.real[m,position] == 0:
                        self.real[m,position] = layer_cnt + 1
                        self.draft[m,position,:] = 0

        # col
        copy_draft = self.draft.transpose([1,0,2]).copy()
        add = np.array([[mapping[real] for real in row] for row in self.real])
        draft = copy_draft + add.transpose([1,0,2])
        for n, col in enumerate(draft):
            for layer_cnt in range(9):
                if col[:, layer_cnt].sum() == 1:
                    position = np.nonzero(col[:, layer_cnt].flatten())[0][0]
                    if self.real[position,n] == 0:
                        self.real[position,n] = layer_cnt + 1
                        self.draft[position,n,:] = 0

    def straight_delete(self, row_unit, col_unit, layer, index, row_straight=True):
            mapping = {0: np.array([3,4,5,6,7,8]),
                       1: np.array([0,1,2,6,7,8]),
                       2: np.array([0,1,2,3,4,5])}
            if row_straight:
                col = mapping[row_unit]
                row = col_unit*3 + index
                self.draft[row, col, layer] = 0
            else:
                row = mapping[col_unit]
                col = row_unit*3 + index
                self.draft[row, col, layer] = 0

    def straight_filter(self):
        for m, unit_row in enumerate(self.draft_units):
            for n, unit in enumerate(unit_row):
                for layer_cnt in range(9):
                    summation_row = unit[:,:,layer_cnt].sum(axis=1)
                    summation_col = unit[:,:,layer_cnt].sum(axis=0)
                    total = summation_col.sum()
                    if summation_row.max() == total and total > 0:
                        row_index = np.nonzero(summation_row)[0][0]
                        self.straight_delete(m, n, layer_cnt, row_index, row_straight=True)
                    if summation_col.max() == total and total > 0:
                        col_index = np.nonzero(summation_col)[0][0]
                        self.straight_delete(m, n, layer_cnt, col_index, row_straight=False)

    def inspect(self):
        for row in self.real:
            real = row[row>0]
            if len(np.unique(real)) != len(real):
                # print('row')
                return False
        for col in self.real.T:
            real = col[col>0]
            if len(np.unique(real)) != len(real):
                # print('col')
                return False
        return True

    def close_filter(self):
        # n=2
        for m, unit_row in enumerate(self.draft_units):
            for n, unit in enumerate(unit_row):
                chosen = (unit.sum(axis=2) <= 2) & (unit.sum(axis=2) > 0)
                if (chosen*1).sum() < 2:
                    continue
                chosen_cell = unit[chosen]
                position = np.argwhere(chosen)
                for i in range(len(chosen_cell)-1):
                    for j in range(i+1, len(chosen_cell)):
                        summation = chosen_cell[i] + chosen_cell[j]
                        if (summation>0).sum() == 2:
                            # print(chosen_cell[i], chosen_cell[j])
                            coverage = np.zeros([3,3])
                            coverage[position[i][0], position[i][1]] = 1
                            coverage[position[j][0], position[j][1]] = 1
                            coverage = coverage == 1
                            layer_array = np.nonzero(summation.flatten())[0]
                            for layer in layer_array:
                                unit[:,:,layer][~coverage] = 0

        for m, row in enumerate(self.draft):
            chosen = (row.sum(axis=1)<=2) & (row.sum(axis=1) > 0)
            if (chosen*1).sum() < 2:
                continue
            chosen_cell = row[chosen]
            position = np.argwhere(chosen)
            for i in range(len(chosen_cell)-1):
                for j in range(i+1, len(chosen_cell)):
                    summation = chosen_cell[i] + chosen_cell[j]
                    if (summation>0).sum() == 2:
                        coverage = np.zeros(9)
                        coverage[position[i][0]] = 1
                        coverage[position[j][0]] = 1
                        coverage = coverage == 1
                        layer_array = np.nonzero(summation.flatten())[0]
                        for layer in layer_array:
                            row[:, layer][~coverage] = 0

        for m, col in enumerate(self.draft.transpose([1,0,2])):
            chosen = (col.sum(axis=1)<=2) & (col.sum(axis=1) > 0)
            if (chosen*1).sum() < 2:
                continue
            chosen_cell = col[chosen]
            position = np.argwhere(chosen)
            for i in range(len(chosen_cell)-1):
                for j in range(i+1, len(chosen_cell)):
                    summation = chosen_cell[i] + chosen_cell[j]
                    if (summation>0).sum() == 2:
                        coverage = np.zeros(9)
                        coverage[position[i][0]] = 1
                        coverage[position[j][0]] = 1
                        coverage = coverage == 1
                        layer_array = np.nonzero(summation.flatten())[0]
                        for layer in layer_array:
                            col[:, layer][~coverage] = 0
        
        # n=3             
        for m, unit_row in enumerate(self.draft_units):
            for n, unit in enumerate(unit_row):
                chosen = (unit.sum(axis=2) <= 3) & (unit.sum(axis=2) > 0)
                if (chosen*1).sum() < 3:
                    continue
                chosen_cell = unit[chosen]
                position = np.argwhere(chosen)
                for i in range(len(chosen_cell)-2):
                    for j in range(i+1, len(chosen_cell)-1):
                        for k in range(j+1, len(chosen_cell)):
                            summation = chosen_cell[i] + chosen_cell[j] + chosen_cell[k]
                            if (summation>0).sum() == 3:
                                # print(chosen_cell[i], chosen_cell[j])
                                coverage = np.zeros([3,3])
                                coverage[position[i][0], position[i][1]] = 1
                                coverage[position[j][0], position[j][1]] = 1
                                coverage[position[k][0], position[k][1]] = 1
                                coverage = coverage == 1
                                layer_array = np.nonzero(summation.flatten())[0]
                                for layer in layer_array:
                                    unit[:,:,layer][~coverage] = 0
                                    
        for m, col in enumerate(self.draft.transpose([1,0,2])):
            chosen = (col.sum(axis=1)<=3) & (col.sum(axis=1) > 0)
            if (chosen*1).sum() < 3:
                continue
            chosen_cell = col[chosen]
            position = np.argwhere(chosen)
            for i in range(len(chosen_cell)-2):
                for j in range(i+1, len(chosen_cell)-1):
                    for k in range(j+1, len(chosen_cell)):
                        summation = chosen_cell[i] + chosen_cell[j] + chosen_cell[k]
                        if (summation>0).sum() == 3:
                            coverage = np.zeros(9)
                            coverage[position[i][0]] = 1
                            coverage[position[j][0]] = 1
                            coverage[position[k][0]] = 1
                            coverage = coverage == 1
                            layer_array = np.nonzero(summation.flatten())[0]
                            for layer in layer_array:
                                col[:, layer][~coverage] = 0
        
        for m, row in enumerate(self.draft):
            chosen = (row.sum(axis=1)<=3) & (row.sum(axis=1) > 0)
            if (chosen*1).sum() < 3:
                continue
            chosen_cell = row[chosen]
            position = np.argwhere(chosen)
            for i in range(len(chosen_cell)-2):
                for j in range(i+1, len(chosen_cell)-1):
                    for k in range(j+1, len(chosen_cell)):
                        summation = chosen_cell[i] + chosen_cell[j] + chosen_cell[k]
                        if (summation>0).sum() == 3:
                            coverage = np.zeros(9)
                            coverage[position[i][0]] = 1
                            coverage[position[j][0]] = 1
                            coverage[position[k][0]] = 1
                            coverage = coverage == 1
                            layer_array = np.nonzero(summation.flatten())[0]
                            for layer in layer_array:
                                row[:, layer][~coverage] = 0

    def guess_iteration(self):
        self.guess_cnt += 1
        ret = False
        possible = [0]
        position = 0
        for m, row in enumerate(self.draft):
            for n, cell in enumerate(row):
                if cell.sum() == 2:
                    possible = np.nonzero(cell.flatten())[0]
                    position = [m,n]
                    ret = True
                    break
            if ret:
                break
        if len(possible) == 1:
            return 0
            # for m, row in enumerate(self.draft):
            #     for n, cell in enumerate(row):
            #         if cell.sum() > 2:
            #             possible = np.nonzero(cell.flatten())[0]
            #             position = [m,n]
            #             ret = True
            #             break
            #     if ret:
            #         break
        print(position, possible)
        for trial in possible:
            new = self.my_copy()
            new.real[position[0], position[1]] = trial+1
            new.draft[position[0],position[1],:] = 0

            if new.cycle_solve():
                print(new,'\n','End: Successfully solved in',
                  new.iter_cnt, 'iterations with ',new.guess_cnt,'guesses.')
                return True
            elif new.inspect():
                print('elif')
                # print(new)
                new.guess_iteration()
            else:
                print('else')
                continue
        # print(new, '\n','End: Failed in', new.iter_cnt,
        #       'iterations with ',new.guess_cnt,'guesses.')
        return False

    def cycle_solve(self):
        previous = (self.real == 0).sum().sum()
        follow = 81

        while follow != previous and previous > 0:
            self.iter_cnt += 1
            # self.simple_filter()
            self.exclusive_fill()
            self.unique_fill()
            follow = previous
            previous = (self.real == 0).sum().sum()
        if previous == 0:
            # print(self,'\n','End: Successfully solved in ',
            #       self.iter_cnt, ' iterations.')
            return True
        if follow == previous:
            # print(self, '\n','End: Failed in',
            #       self.iter_cnt, 'iterations.')
            return False

    def run(self):
        if self.cycle_solve():
            print(self,'\n','End: Successfully solved in ',
                  self.iter_cnt, 'iterations with',self.guess_cnt,'guesses.')
        else:
            self.guess_iteration()



In [13]:
table = Table(seed=17)
print(table)

[[8 1 2 7 5 3 6 4 9]
 [9 0 3 6 0 0 0 0 0]
 [0 7 0 0 9 0 2 0 0]
 [0 5 0 0 0 7 0 0 0]
 [0 0 0 0 4 5 7 0 0]
 [0 0 0 1 0 0 0 3 0]
 [0 0 1 0 0 0 0 6 8]
 [0 0 8 5 0 0 0 1 0]
 [0 9 0 0 0 0 4 0 0]]
zero count: 51


In [14]:
# table.cycle_solve()
table.run()


[1, 4] [1 7]
[[8 1 2 7 5 3 6 4 9]
 [9 4 3 6 2 8 1 7 5]
 [6 7 5 4 9 1 2 8 3]
 [1 5 4 9 3 7 8 2 6]
 [2 8 6 3 4 5 7 9 1]
 [7 6 9 1 8 2 5 3 4]
 [5 3 1 2 7 4 9 6 8]
 [4 2 8 5 6 9 3 1 7]
 [3 9 7 8 1 6 4 5 2]]
zero count: 0 
 End: Successfully solved in 5 iterations with  1 guesses.
